# Chapter 04. 머신러닝 회귀 문제 연습하기 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 실습 가이드
1. 데이터를 다운로드하여 Colab에 불러옵니다.
> 다운로드한 데이터의 위치는 **Colab Notebooks/data/** 로 통일합니다.

2. 필요한 라이브러리는 모두 코드로 작성되어 있습니다.
3. 코드는 위에서부터 아래로 순서대로 실행합니다.
4. 전체 문제 구성은 좌측 첫 번째 아이콘을 통해 확인할 수 있습니다.

### <b> 데이터의 속성</b>
**시작 전에** 원본 Kaggle 페이지에서 상세한 데이터에 대한 설명을 확인하세요 !

Reference : https://www.kaggle.com/c/zillow-prize-1/data

### <b>학습목표</b>
- 1) Regression 모델의 이해
- 2) 학습했던 다양한 머신러닝 회귀모델의 이해
- 3) 회귀를 위한 데이터 전처리 방법에 대한 이해
- 4) feature engineering에 대한 이해
- 5) 평가결과를 바탕으로 모델을 개선하는 방법 습득

- 출제자: 김용담 강사

## ++ 추가 설명 ++

원래 Kaggle 대회의 목적은 2016, 2017년 데이터를 통해서 2017년 10, 11월 집값을 예측하는 문제를 풉니다.

저희는 실습의 편의성을 위해서 문제를 간소화하여 2016년도 데이터를 가지고 2016년 데이터 중에서 2016년 1월부터 9월까지의 데이터는 학습 데이터로 사용하고, 10월부터 12월까지의 3개월 데이를 테스트 데이터로 사용하는 문제로 변환합니다.

그렇기 때문에, 데이터 중에서 2016년 데이터만 사용합니다!

## Step 1. 예측할 데이터 불러오기

### 문제 01. 데이터 불러오기

In [ ]:
# 분석에 필요한 라이브러리를 불러옵니다
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/data/zillow-prize-1'
train = pd.read_csv(base_path + "/train_2016_v2.csv")
properties = pd.read_csv(base_path + "/properties_2016.csv")

### 문제 02. 데이터 미리보기

In [ ]:
# 데이터 크기 확인
print(train.shape, properties.shape)

# 데이터 일부 확인
display(train.head())
display(properties.head())

(90275, 3) (2985217, 58)


,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,010D,269.0,NaN,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,0109,261.0,LCA11*,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037703e+07,51617.0,3101.0,NaN,96095.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,1210,31.0,LAM1,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1947.0,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


### 문제 03. 데이터 확인하기

In [ ]:
# train 데이터에서 결측치가 있는 column 확인하기
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   parcelid         90275 non-null  int64  
 1   logerror         90275 non-null  float64
 2   transactiondate  90275 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [ ]:
# test도 확인
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      int64  
 1   airconditioningtypeid         float64
 2   architecturalstyletypeid      float64
 3   basementsqft                  float64
 4   bathroomcnt                   float64
 5   bedroomcnt                    float64
 6   buildingclasstypeid           float64
 7   buildingqualitytypeid         float64
 8   calculatedbathnbr             float64
 9   decktypeid                    float64
 10  finishedfloor1squarefeet      float64
 11  calculatedfinishedsquarefeet  float64
 12  finishedsquarefeet12          float64
 13  finishedsquarefeet13          float64
 14  finishedsquarefeet15          float64
 15  finishedsquarefeet50          float64
 16  finishedsquarefeet6           float64
 17  fips                          float64
 18  fireplacecnt          

## Step 2. Data Preprocessing

- 학습에 필요한 데이터를 만들기 위해서, train 데이터와 properties 데이터를 합칩니다.

- 합친 다음, 결측치가 존재한다면 결측치를 처리합니다.

### 문제 04. train data와 properties data 합치기

In [ ]:
# train data와 properties data에 공통으로 있는 column을 기준으로 합칩니다.
data = pd.merge(train, properties, how="left", on="parcelid")
data

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,NaN,NaN,1684.0,1684.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34280990.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LARS,6.037107e+07,12447.0,3101.0,31817.0,96370.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,2263.0,2263.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,468.0,NaN,NaN,33668120.0,-117677556.0,3643.0,NaN,NaN,NaN,NaN,NaN,1,261.0,NaN,6.059052e+07,32380.0,1286.0,NaN,96962.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,NaN,NaN,2217.0,2217.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,PSR6,6.037464e+07,47019.0,3101.0,275411.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,NaN,839.0,839.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,010C,266.0,LAR3,6.037296e+07,12447.0,3101.0,54300.0,96222.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,NaN,2283.0,2283.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,598.0,NaN,NaN,33485643.0,-117700234.0,6000.0,1.0,NaN,NaN,NaN,1.0,122,261.0,NaN,6.059042e+07,17686.0,1286.0,NaN,96961.0,8.0,NaN,1.0,NaN,NaN,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,2016-12-30,1.0,NaN,NaN,1.0,1.0,NaN,4.0,1.0,NaN,NaN,653.0,653.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,2.0,34203000.0,-118562000.0,93676.0,1.0,NaN,NaN,NaN,1.0,010C,266.0,LARD1.5,6.037132e+07,12447.0,3101.0,40548.0,96364.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1979.0,NaN,NaN,43800.0,191000.0,2015.0,147200.0,2495.24,NaN,NaN,6.037132e+13
90271,12046695,0.0070,2016-12-30,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,NaN,NaN,2856.0,2856.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34198744.0,-118224849.0,9343.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,GLR1RY,6.037301e+07,45457.0,3101.0,274580.0,96327.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1965.0,NaN,NaN,117893.0,161111.0,2015.0,43218.0,1886.54,NaN,NaN,6.037301e+13
90272,12995401,-0.2679,2016-12-30,NaN,NaN,NaN,2.0,4.0,NaN,7.0,2.0,NaN,NaN,2617.0,NaN,NaN,2617.0,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,NaN,34075655.0,-118025537.0,6865.0,NaN,NaN,NaN,NaN,NaN,0200,246.0,EMR3YY,6.037433e+07,51861.0,3101.0,NaN,96478.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,1924.0,NaN,NaN,22008.0,38096.0,2015.0,16088.0,1925.70,Y,14.0,6.037433e+1

### 문제 05. 합친 data에서 결측치가 있는 데이터 뽑기

In [ ]:
# 결측치가 있는 데이터를 뽑습니다.
missing_rows = data[data.isnull().any(axis=1)]
missing_rows

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,NaN,NaN,1684.0,1684.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34280990.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LARS,6.037107e+07,12447.0,3101.0,31817.0,96370.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,2263.0,2263.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,468.0,NaN,NaN,33668120.0,-117677556.0,3643.0,NaN,NaN,NaN,NaN,NaN,1,261.0,NaN,6.059052e+07,32380.0,1286.0,NaN,96962.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,NaN,NaN,2217.0,2217.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,PSR6,6.037464e+07,47019.0,3101.0,275411.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,NaN,839.0,839.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,010C,266.0,LAR3,6.037296e+07,12447.0,3101.0,54300.0,96222.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,NaN,2283.0,2283.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,598.0,NaN,NaN,33485643.0,-117700234.0,6000.0,1.0,NaN,NaN,NaN,1.0,122,261.0,NaN,6.059042e+07,17686.0,1286.0,NaN,96961.0,8.0,NaN,1.0,NaN,NaN,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,2016-12-30,1.0,NaN,NaN,1.0,1.0,NaN,4.0,1.0,NaN,NaN,653.0,653.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,2.0,34203000.0,-118562000.0,93676.0,1.0,NaN,NaN,NaN,1.0,010C,266.0,LARD1.5,6.037132e+07,12447.0,3101.0,40548.0,96364.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1979.0,NaN,NaN,43800.0,191000.0,2015.0,147200.0,2495.24,NaN,NaN,6.037132e+13
90271,12046695,0.0070,2016-12-30,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,NaN,NaN,2856.0,2856.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34198744.0,-118224849.0,9343.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,GLR1RY,6.037301e+07,45457.0,3101.0,274580.0,96327.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1965.0,NaN,NaN,117893.0,161111.0,2015.0,43218.0,1886.54,NaN,NaN,6.037301e+13
90272,12995401,-0.2679,2016-12-30,NaN,NaN,NaN,2.0,4.0,NaN,7.0,2.0,NaN,NaN,2617.0,NaN,NaN,2617.0,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,NaN,34075655.0,-118025537.0,6865.0,NaN,NaN,NaN,NaN,NaN,0200,246.0,EMR3YY,6.037433e+07,51861.0,3101.0,NaN,96478.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,1924.0,NaN,NaN,22008.0,38096.0,2015.0,16088.0,1925.70,Y,14.0,6.037433e+1

## 결측치 채우기

- 결측치가 포함된 row가 모든 row이므로, 특정 column에 결측치가 많이 포함되어 있는 경우입니다.

- 결측치가 많이 포함된 column들을 먼저 찾고, 해당 column들을 어떻게 처리할지 결정합니다.

### 문제 06. 결측치 패턴 파악하기

In [ ]:
# 결측치 패턴 확인하기
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90275 entries, 0 to 90274
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      90275 non-null  int64  
 1   logerror                      90275 non-null  float64
 2   transactiondate               90275 non-null  object 
 3   airconditioningtypeid         28781 non-null  float64
 4   architecturalstyletypeid      261 non-null    float64
 5   basementsqft                  43 non-null     float64
 6   bathroomcnt                   90275 non-null  float64
 7   bedroomcnt                    90275 non-null  float64
 8   buildingclasstypeid           16 non-null     float64
 9   buildingqualitytypeid         57364 non-null  float64
 10  calculatedbathnbr             89093 non-null  float64
 11  decktypeid                    658 non-null    float64
 12  finishedfloor1squarefeet      6856 non-null   float64
 13  c

- basementsqft, buildingclasstypeid 같은 column들에 값이 거의 없습니다.

- 90% 이상 결측인 column들을 찾아서 제거합니다.

### 문제 07. 90% 이상 결측인 column들 제거하기

In [ ]:
# 90% 이상 결측인 column들 뽑기
missing_cols = data.columns[data.isnull().mean() > 0.9]
missing_cols

Index(['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid',
       'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13',
       'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6',
       'hashottuborspa', 'poolsizesum', 'pooltypeid10', 'pooltypeid2',
       'storytypeid', 'typeconstructiontypeid', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'fireplaceflag', 'taxdelinquencyflag',
       'taxdelinquencyyear'],
      dtype='object')

In [ ]:
# 90% 이상인 column들을 제거합니다.
data.drop(missing_cols, axis=1, inplace=True)
data

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,2.0,3.0,4.0,2.0,1684.0,1684.0,6037.0,NaN,2.0,NaN,NaN,2.0,34280990.0,-118488536.0,7528.0,NaN,NaN,0100,261.0,LARS,6.037107e+07,12447.0,3101.0,31817.0,96370.0,0.0,NaN,1.0,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,3.5,4.0,NaN,3.5,2263.0,2263.0,6059.0,NaN,3.0,2.0,468.0,NaN,33668120.0,-117677556.0,3643.0,NaN,NaN,1,261.0,NaN,6.059052e+07,32380.0,1286.0,NaN,96962.0,0.0,1.0,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN
2,12098116,-0.0040,2016-01-01,1.0,3.0,2.0,4.0,3.0,2217.0,2217.0,6037.0,NaN,3.0,NaN,NaN,2.0,34136312.0,-118175032.0,11423.0,NaN,NaN,0100,261.0,PSR6,6.037464e+07,47019.0,3101.0,275411.0,96293.0,0.0,NaN,1.0,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,2.0,2.0,4.0,2.0,839.0,839.0,6037.0,NaN,2.0,NaN,NaN,2.0,33755800.0,-118309000.0,70859.0,NaN,NaN,010C,266.0,LAR3,6.037296e+07,12447.0,3101.0,54300.0,96222.0,0.0,NaN,1.0,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,2.5,4.0,NaN,2.5,2283.0,2283.0,6059.0,NaN,2.0,2.0,598.0,NaN,33485643.0,-117700234.0,6000.0,1.0,1.0,122,261.0,NaN,6.059042e+07,17686.0,1286.0,NaN,96961.0,8.0,1.0,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,2016-12-30,1.0,1.0,1.0,4.0,1.0,653.0,653.0,6037.0,NaN,1.0,NaN,NaN,2.0,34203000.0,-118562000.0,93676.0,1.0,1.0,010C,266.0,LARD1.5,6.037132e+07,12447.0,3101.0,40548.0,96364.0,0.0,NaN,1.0,1979.0,NaN,43800.0,191000.0,2015.0,147200.0,2495.24,6.037132e+13
90271,12046695,0.0070,2016-12-30,NaN,3.0,3.0,4.0,3.0,2856.0,2856.0,6037.0,NaN,3.0,NaN,NaN,2.0,34198744.0,-118224849.0,9343.0,NaN,NaN,0100,261.0,GLR1RY,6.037301e+07,45457.0,3101.0,274580.0,96327.0,0.0,NaN,1.0,1965.0,NaN,117893.0,161111.0,2015.0,43218.0,1886.54,6.037301e+13
90272,12995401,-0.2679,2016-12-30,NaN,2.0,4.0,7.0,2.0,2617.0,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34075655.0,-118025537.0,6865.0,NaN,NaN,0200,246.0,EMR3YY,6.037433e+07,51861.0,3101.0,NaN,96478.0,0.0,NaN,2.0,1924.0,NaN,22008.0,38096.0,2015.0,16088.0,1925.70,6.037433e+13
90273,11402105,0.0602,2016-12-30,NaN,2.0,2.0,4.0,2.0,1034.0,1034.0,6037.0,NaN,2.0,NaN,NaN,2.0,33963900.0,-118367000.0,20033.0,NaN,NaN,010C,266.0,INR4*,6.037601e+07,45888.0,3101.0,NaN,96133.0,0.0,NaN,1.0,1981.0,NaN,132991.0,165869.0,2015.0,32878.0,2285.57,6.037601e+13


### 문제 08. 결측이 존재하는 나머지 column들 찾기

- 나머지 결측을 포함하고 있는 column들이 여전히 많습니다.

In [ ]:
# 여전히 결측치를 포함하고 있는 column들 찾기
missing_cols = data.columns[data.isnull().any()]
missing_cols

Index(['airconditioningtypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'lotsizesquarefeet', 'poolcnt', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc', 'regionidcity',
       'regionidneighborhood', 'regionidzip', 'threequarterbathnbr', 'unitcnt',
       'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount',
       'censustractandblock'],
      dtype='object')

### 문제 09. 결측치 채우기 1

- 나머지 column들중 continuous한 numeric feature들은 평균값으로 채워줍니다.

- column들중 매우 높은 cardinality를 가지고 있는 column들은 continuous할 가능성이 높습니다.

In [ ]:
# continuous feature를 찾기 위해서, column별로 cardinality를 구합니다.
percentages = []
for col in missing_cols:
    print("%s,\t %f%%" % (col, data[col].nunique() / len(data[col])))
    percentage = data[col].nunique() / len(data[col])
    percentages.append(percentage)

missing_df = pd.DataFrame({"column" : missing_cols, "percentage" : percentages})
missing_df

airconditioningtypeid,	 0.000066%
buildingqualitytypeid,	 0.000089%
calculatedbathnbr,	 0.000244%
calculatedfinishedsquarefeet,	 0.056516%
finishedsquarefeet12,	 0.055187%
fireplacecnt,	 0.000055%
fullbathcnt,	 0.000155%
garagecarcnt,	 0.000166%
garagetotalsqft,	 0.009637%
heatingorsystemtypeid,	 0.000133%
lotsizesquarefeet,	 0.221723%
poolcnt,	 0.000011%
pooltypeid7,	 0.000011%
propertycountylandusecode,	 0.000853%
propertyzoningdesc,	 0.022110%
regionidcity,	 0.001961%
regionidneighborhood,	 0.005472%
regionidzip,	 0.004298%
threequarterbathnbr,	 0.000044%
unitcnt,	 0.000111%
yearbuilt,	 0.001440%
numberofstories,	 0.000044%
structuretaxvaluedollarcnt,	 0.614234%
taxvaluedollarcnt,	 0.619640%
landtaxvaluedollarcnt,	 0.632135%
taxamount,	 0.942786%
censustractandblock,	 0.469654%


,column,percentage
0,airconditioningtypeid,0.000066
1,buildingqualitytypeid,0.000089
2,calculatedbathnbr,0.000244
3,calculatedfinishedsquarefeet,0.056516
4,finishedsquarefeet12,0.055187
5,fireplacecnt,0.000055
6,fullbathcnt,0.000155
7,garagecarcnt,0.000166
8,garagetotalsqft,0.009637
9,heatingorsystemtypeid,0.000133


- cardinality가 눈에 띄게 높은 structuretaxvaluedollarcnt, taxvaluedollarcnt, landtaxvaluedollarcnt, taxamount, censustractandblock column은 평균값으로 채웁니다.

In [ ]:
# # continuous column 지정
continuous = missing_df.column[missing_df.percentage > 0.1].values
continuous

array(['lotsizesquarefeet', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount',
       'censustractandblock'], dtype=object)

### 문제 10. 결측치 채우기 2

- discrete한 numeric feature들은 최빈값으로 결측치를 채워줍니다.

In [ ]:
# discrete feature를 찾기 위해서, column별로 cardinality를 구합니다.
missing_df

,column,percentage
0,airconditioningtypeid,0.000066
1,buildingqualitytypeid,0.000089
2,calculatedbathnbr,0.000244
3,calculatedfinishedsquarefeet,0.056516
4,finishedsquarefeet12,0.055187
5,fireplacecnt,0.000055
6,fullbathcnt,0.000155
7,garagecarcnt,0.000166
8,garagetotalsqft,0.009637
9,heatingorsystemtypeid,0.000133


- 앞에서 찾은 5개의 column을 제외한 나머지 column들을 discrete feature로 지정합니다.

In [ ]:
# continuous column 지정
discretes = np.setdiff1d(missing_df.column, continuous)
discretes

array(['airconditioningtypeid', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid',
       'numberofstories', 'poolcnt', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc', 'regionidcity',
       'regionidneighborhood', 'regionidzip', 'threequarterbathnbr',
       'unitcnt', 'yearbuilt'], dtype=object)

### 문제 11. 결측치 채우기 3

In [ ]:
# continuous feature는 평균값으로, discrete feature는 최빈값으로 결측치를 채웁니다.

modes = data[discretes].mode() # 최빈값
means = data[continuous].mean() # 평균

for column in discretes:
    data[column].fillna(modes[column].values[0], inplace=True)
    
for column in continuous:
    data[column].fillna(means[column], inplace=True)

In [ ]:
data[data.isnull().any(axis=1)]

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock


## Step 3. Feature Engineering

- 분석에 사용할 feature들을 만들고, 학습을 위한 데이터를 준비하는 단계입니다.

- 모든 categorical feature들을 변환하여 데이터를 feature vector로 만들어봅니다.

### 문제 12. date_time을 시간정보로 변환하기

- date_time feature로 사용하기 위해서 시간 정보로 변환해줍니다.

In [ ]:
# datetime data type으로 변환하기
data.transactiondate = pd.to_datetime(data.transactiondate)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90275 entries, 0 to 90274
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   parcelid                      90275 non-null  int64         
 1   logerror                      90275 non-null  float64       
 2   transactiondate               90275 non-null  datetime64[ns]
 3   airconditioningtypeid         90275 non-null  float64       
 4   bathroomcnt                   90275 non-null  float64       
 5   bedroomcnt                    90275 non-null  float64       
 6   buildingqualitytypeid         90275 non-null  float64       
 7   calculatedbathnbr             90275 non-null  float64       
 8   calculatedfinishedsquarefeet  90275 non-null  float64       
 9   finishedsquarefeet12          90275 non-null  float64       
 10  fips                          90275 non-null  float64       
 11  fireplacecnt                

### 문제 13. 필요없는 feature 제거하기

In [ ]:
# 분석에 사용할 수 없는 두 개의 object type column은 제거합니다.
data.drop(["propertycountylandusecode", "propertyzoningdesc"], axis=1, inplace=True)

## 문제 14. train-test 분리하기

In [ ]:
# 2016년 10월 전후로 데이터를 나눕니다.
X_train_2016 = data.loc[data.transactiondate < '2016-10-01', :] 
X_test_2016 = data.loc[data.transactiondate >= '2016-10-01', :]
print(X_train_2016.shape, X_test_2016.shape)

(81733, 38) (8542, 38)


### 문제 15. X, y 분리하기

In [ ]:
# 2016-10-01 이전 데이터 X, y 분리
X_train = X_train_2016.drop(columns=["parcelid", "transactiondate", "logerror"])
y_train = X_train_2016.logerror

# 2016-10-01 이후 데이터 X, y 분리
X_test = X_test_2016.drop(columns=["parcelid", "transactiondate", "logerror"])
y_test = X_test_2016.logerror

### 문제 16. feature scaling

- tree method 계열은 scaling이 필요하지 않지만, linear regression 계열은 scale에 굉장히 민감하기 때문에, 사전에 scaling을 해줍니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 스케일 조정된 X_train 데이터 확인하기
X_train_scaled 

array([[-1.52003923e-01, -2.80058904e-01, -2.83676527e-02, ...,
        -1.05502370e-01,  1.08119558e-01, -5.92933769e-01],
       [-1.52003923e-01,  1.21218681e+00,  8.34196840e-01, ...,
        -1.01360964e-01,  6.14103366e-01, -2.04074194e-03],
       [-1.52003923e-01,  7.14771571e-01, -8.90932146e-01, ...,
        -5.54686007e-01,  8.11255722e-01, -5.75411661e-01],
       ...,
       [-1.52003923e-01,  7.14771571e-01, -2.83676527e-02, ...,
        -6.58829228e-01, -6.63929460e-01, -5.74582448e-01],
       [-1.52003923e-01, -2.80058904e-01,  8.34196840e-01, ...,
         1.70450389e-01,  1.03982417e-01, -5.92710034e-01],
       [-1.52003923e-01, -2.80058904e-01,  8.34196840e-01, ...,
        -5.65273493e-01, -6.06841662e-01,  3.03253106e+00]])

### 참고: scikit-learn에서 제공하는 피처 스케일러(scaler)

- `StandardScaler`: 기본 스케일, 각 피처의 평균을 0, 표준편차를 1로 변환
- `RobustScaler`: 위와 유사하지만 평균 대신 중간값(median)과 일분위, 삼분위값(quartile)을 사용하여 이상치 영향을 최소화
- `MinMaxScaler`: 모든 피처의 최대치와 최소치가 각각 1, 0이 되도록 스케일 조정
- `Normalizer`: 피처(컬럼)이 아니라 row마다 정규화되며, 유클리드 거리가 1이 되도록 데이터를 조정하여 빠르게 학습할 수 있게 함

<p> 스케일 조정을 하는 이유는 데이터의 값이 너무 크거나 작을 때 학습이 제대로 되지 않을 수도 있기 때문입니다. 또한 스케일의 영향이 절대적인 분류기(예: knn과 같은 거리기반 알고리즘)의 경우, 스케일 조정을 필수적으로 검토해야 합니다.
    
<p> 반면 어떤 항목은 원본 데이터의 분포를 유지하는 것이 나을 수도 있습니다. 예를 들어, 데이터가 거의 한 곳에 집중되어 있는 feature를 표준화시켜 분포를 같게 만들었을 때, 작은 단위의 변화가 큰 차이를 나타내는 것처럼 학습될 수도 있습니다. 또한 스케일의 영향을 크게 받지 않는 분류기(예: 트리 기반 앙상블 알고리즘)를 사용할 경우에도 성능이 준수하게 나오거나 과대적합(overfitting)의 우려가 적다면 생략할 수도 있습니다.
    
<p> 스케일 조정시 유의해야할 점은 원본 데이터의 의미를 잃어버릴 수 있다는 것입니다. 최종적으로 답을 구하는 것이 목적이 아니라 모델의 해석이나 향후 다른 데이터셋으로의 응용이 더 중요할 때 원 피처에 대한 설명력을 잃어버린다면 모델 개선이 어려울 수도 있습니다. 이 점을 함께 고려하시면 좋겠습니다.

## Step 4. Training

- 공부한 각 머신러닝 회귀 모델들을 돌려보고, 성능을 평가합니다.

- 아래 모델들이 그 대상에 해당합니다.

- Linear Regression / Ridge / Lasso / Random Forest / SVR / xgboost / lightgbm / catboost

### 문제 17. Linear Regression 모델 돌려보기

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

linear_reg = LinearRegression()
linear_reg.fit(X_train_scaled, y_train)

LinearRegression()

### 문제 18. Linear Regression 모델의 MAE 평가하기

In [ ]:
pred = linear_reg.predict(X_train_scaled)
linear_reg_score = mean_absolute_error(y_train, pred)

print('Linear Regression MAE: {0:0.4f}'.format(linear_reg_score))

Linear Regression MAE: 0.0687


### 문제 19. Lasso 모델 돌려보기

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train_scaled, y_train)

Lasso()

### 문제 20. Lasso 평가하기

In [ ]:
pred = lasso.predict(X_train_scaled)
lasso_score = mean_absolute_error(y_train, pred)

print('Lasso MAE: {0:0.4f}'.format(lasso_score))

Lasso MAE: 0.0687


### 문제 21. Random Forest 모델 돌려보기

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=5, max_features=0.85)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(max_depth=5, max_features=0.85)

In [ ]:
pred = rf.predict(X_train_scaled) 
rf_score = mean_absolute_error(y_train, pred)
print('Random Forest MAE : {0:0.4f}'.format(rf_score))

Random Forest MAE : 0.0682


### 문제 22. Support Vector Machine 모델 돌려보기

In [ ]:
from sklearn.svm import LinearSVR

# 데이터가 10000개 이상일 때는 LinearSVR을 추천합니다.
svr = LinearSVR()
svr.fit(X_train_scaled, y_train)

LinearSVR()

In [ ]:
pred = svr.predict(X_train_scaled) 
svm_score = mean_absolute_error(y_train, pred)
print('Support Vector Machine MAE: {0:0.4f}'.format(svm_score))

Support Vector Machine MAE: 0.0730


### 문제 23. XGBoost 모델 돌려보기

- xgboost의 구현체가 많지만, 여기서는 sklearn version을 사용합니다.

In [ ]:
from xgboost.sklearn import XGBRegressor

xgb = XGBRegressor(max_depth=6, max_features=0.85)
xgb.fit(X_train_scaled, y_train)

[17:39:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(max_depth=6, max_features=0.85)

In [ ]:
pred = xgb.predict(X_train_scaled) 
xgb_score = mean_absolute_error(y_train, pred)
print('XGBoost MAE : {0:0.4f}'.format(xgb_score))

XGBoost MAE : 0.0664


### 문제 24. Light GBM 모델 돌려보기

In [ ]:
from lightgbm.sklearn import LGBMRegressor

lgb = LGBMRegressor(max_depth=6, max_features=0.85)
lgb.fit(X_train_scaled, y_train)

LGBMRegressor(max_depth=6, max_features=0.85)

In [ ]:
pred = lgb.predict(X_train_scaled) 
lgb_score = mean_absolute_error(y_train, pred)
print('LightGBM MAE: {0:0.4f}'.format(lgb_score))

LightGBM MAE: 0.0672


### 참고 : 부스팅(Boosting) 모델 개요


- 부스팅은 여러 트리의 적합 결과를 합하는 앙상블 알고리즘의 하나로, 이 때 sequential의 개념이 추가되어 있습니다. 즉 연속적인 weak learner, 바로 직전 weak learner의 error를 반영한 현재 weak learner를 잡겠다는 것입니다. 이 아이디어는 Gradient Boosting Model(GBM)에서 loss를 계속 줄이는 방향으로 weak learner를 잡는다는 개념으로 확장됩니다.

![boost](https://pluralsight2.imgix.net/guides/81232a78-2e99-4ccc-ba8e-8cd873625fdf_2.jpg)


- 부스팅 계열 모델은 XGBoost, LightGBM, CatBoost 등이 있습니다.


- 더 자세한 내용은 다음 Step에서 살펴보겠습니다.

### 문제 25. Test prediction

- 학습한 모델을 기반하여 test 데이터를 평가합니다. 그 예측값이 대회 제출이 되는 기준입니다.


- 실제 현업에서는 서비스에 들어가는 기술을 test 데이터로 평가합니다. 마치 유저가 들어온 것처럼요.

- 이번엔 데이터로 2016년도 데이터만 사용했기 때문에, 최종 데이터 template이 아닌 예측한 데이터만 정리합니다.

In [ ]:
# 모델을 하나 정해서, test data에 대한 inference를 진행합니다.
final = lgb.predict(X_test_scaled)
final

array([ 0.01399858,  0.0058371 , -0.01593919, ..., -0.04053807,
        0.00570631,  0.0217139 ])

In [ ]:
# 결과 파일 정리
result = pd.DataFrame({"id" : X_test_2016.parcelid, "logerror" : final})
result

,id,logerror
3161,13927813,0.013999
3421,13923841,0.005837
4191,12032773,-0.015939
10829,10736972,0.040786
10941,11663348,0.023256
...,...,...
90270,10774160,0.006915
90271,12046695,0.026019
90272,12995401,-0.040538
90273,11402105,0.005706


### 문제 26. 최종 성능 정리

In [ ]:
print('MAE Comparisons for Various Regression Models\n')
print('linear_reg_score:', '{0:0.5f}'.format(linear_reg_score))
print('lasso_score   :', '{0:0.5f}'.format(lasso_score))
print('rf_score   :', '{0:0.5f}'.format(rf_score))
print('svm_score   :', '{0:0.5f}'.format(svm_score))
print('xgb_score   :', '{0:0.5f}'.format(xgb_score))
print('lgb_score   :', '{0:0.5f}'.format(lgb_score))

MAE Comparisons for Various Regression Models

linear_reg_score: 0.06870
lasso_score   : 0.06868
rf_score   : 0.06823
svm_score   : 0.07303
xgb_score   : 0.06636
lgb_score   : 0.06718
